In [321]:
"""
Hackathon - INCAP - IconPro GmbH
Timeseries Classification with Transformers
"""
import pandas as pd
from tensorflow import keras
from dataclasses import dataclass
from tensorflow.keras import layers
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
# Import packages as you need

In [322]:
def load_data(data_path):
    """
    Loading of the dataset provided
    Edit the code below
    """
    data = pd.read_pickle(data_path)
    return data

In [323]:
def preprocess_data(data):
    """
    A standard nan removal to be added.
    Add more preprocessing steps if needed.
    """
    
    X = data['dim_0'].apply(lambda x: x.reshape(500,1))
    
    for i in range(data.shape[0]):
        if True in np.isnan(data['dim_0'][i]).flatten():
            print(i)
            
    input_x = []
    for array in X:
        input_x.append(array)
    
#     X = pd.DataFrame(data.dim_0.tolist())
#     X = X.to_numpy()
    
    y = data['labels']
    y = y.astype(int)
    y[y == -1] = 0
    return input_x,y

In [324]:
def split_train_test(X, y):
    """
    Splitting the data into train, test, validation 
    """
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)
    
    return X_train, X_test, X_val, y_train, y_test, y_val

In [325]:
def normalization(X_train, X_test, X_val):
    scaler = MinMaxScaler(feature_range=(0,1))
    X_train = np.reshape(X_train, (-1,500))
    X_val = np.reshape(X_val, (-1,500))
    X_test = np.reshape(X_test, (-1,500))
    
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)
    
    return np.reshape(X_train, (-1,500,1)), np.reshape(X_val, (-1,500,1)), np.reshape(X_test, (-1,500,1))

In [326]:
input_shape = X_train.shape[1:]

In [327]:
def timeseries_transform(data, head_size, num_heads, ff_dim, dropout=0):
    """
    Implement the timeseries transformer here
    """
    # Normalization and Attention
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(data, data)
    
    x = layers.Dropout(dropout)(x)
    res = x + data

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=data.shape[-1], kernel_size=1)(x)
    return x + res

In [328]:
n_classes = 2

In [329]:
def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = timeseries_transform(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)

In [330]:
def model_training(X_train, y_train):
    """
    Train the data with the compatible model
    """
    
    input_shape = X_train.shape[1:]

    model = build_model(input_shape, head_size=256, num_heads=4, ff_dim=4, num_transformer_blocks=4, mlp_units=[128], mlp_dropout=0.4, dropout=0.25)

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=keras.optimizers.Adam(learning_rate=1e-4),
        metrics=["sparse_categorical_accuracy"],
    )
    
    model.summary()

    callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

    model.fit(X_train, y_train, epochs=10, batch_size=128, callbacks=callbacks)
    return model

In [331]:
def metric(y_act, y_pred, model):
    """
    Standard metrics and plotting should be same
    Metrics should be computed on validation data(unseen data)
    1. Balanced accuracy score
    2. Confusion matrix
    3. Per-class accuracy
    """
    
    cm = metrics.confusion_matrix(y_act, y_pred[:,0])
    balanced_accuracy = metrics.balanced_accuracy_score(y_act, y_pred[:,0])
    
    return cm, balanced_accuracy

In [332]:
def validation(X_val, y_val, metrics):
    """
    Comparing the results with provided Series Embedder
    Plot confusion matrices of self analysis and LSTM with balanced_accuracy
    
    """
    
    score = model.evaluate(X_val, y_val, verbose=1)
    
    return score

In [333]:
def evaluate(X_test, y_act, metric):
    y_pred = model.predict(X_test, verbose=1)
    cm, ba = metric(y_act, y_pred[:,0])
    
    return y_pred, cm, ba

In [334]:
path = "../input/fordadata/data.pkl"
data = load_data(path)
X, y = preprocess_data(data)

X_train, X_test, X_val, y_train, y_test, y_val = split_train_test(X, y)
X_train, X_test, X_val = normalization(X_train, X_test, X_val)

In [335]:
model_self=model_training(X_train, y_train)
# metrics=metric(val,model_self)

# lstm_cm,lstm_balanced_accuracy=lstm(preprocessed_data,target='labels')
# metrics_validation = [lstm_cm, lstm_balanced_accuracy]
# validation(metrics,metrics_validation)

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           [(None, 500, 1)]     0                                            
__________________________________________________________________________________________________
multi_head_attention_61 (MultiH (None, 500, 1)       7169        input_22[0][0]                   
                                                                 input_22[0][0]                   
__________________________________________________________________________________________________
dropout_136 (Dropout)           (None, 500, 1)       0           multi_head_attention_61[0][0]    
__________________________________________________________________________________________________
tf.__operators__.add_120 (TFOpL (None, 500, 1)       0           dropout_136[0][0]          